# Primo notebook di test con Perceval by Quandela

Proviamo a implementare delle misure studiate nel corso di AQM francese: esperimento di interferenza con due _beam splitters_, e dopo vediamo se, diõcristõ, funziona.

## Installazione e importazione

Provo a seguire le istruzioni su https://perceval.quandela.net/docs/usage.html?highlight=install, anche se mi danno già problemi.

Dopo aver creato un ambiente virtuale in Conda ed averlo attivato, eseguiamo il comando:

In [1]:
!pip3 install perceval-quandela

In [128]:
import perceval as pcvl
import perceval.lib.phys as phys
import perceval.lib.symb as symb
from math import sqrt
import sympy as sp

Nota: la pagina di documentazione https://perceval.quandela.net/docs/circuits.html riporta che:

- *`symb` provides circuits with generally less parameters. This is useful to build generic optical circuits which implement quantum circuits, and perform symbolic computation.*

- `phys` *provides circuits with more parameters, which allow a sharper modelisation of the components in a physical experiment.*


## Breve introduzione su stati e operatori

### 1. Stati e gradi di libertà
In Perceval in un circuito la prima cosa che si nota è la rappresentazione di quelli che potremmo chiamare "canali". Essi sono detti *modes* nella documentazione, e penso che effettivamente corrispondano a fasci di fotoni diversi.

Un esempio di stato a 4 canali, con 1 fotone nel primo canale, 2 nel secondo e 0 nel terzo e nel quarto, è:

#### 1.1. Basic State

In [2]:
estate = pcvl.BasicState("|1,2,0,0>")
print(estate,"\n")
print("Numero di fotoni totali:", estate.n)
print("Numero di 'modes' (canali):", estate.m)
print("Numero di fotoni nel secondo canale:", estate[1])

|1,2,0,0> 

Numero di fotoni totali: 3
Numero di 'modes' (canali): 4
Numero di fotoni nel secondo canale: 2


Bene, tutto bello. Credo allora che, finché non si agisce su questo stato, la sua espressione matematica possa essere la seguente:
$$\left|\psi_1\otimes\psi_2\otimes\psi_2\right>$$
e i due fotoni nel modo 2 (nello stato $\left|\psi_2\right>$) dovrebbero essere indistinguibili, mentre ciò che rende distinguibili fisicamente i modi 1 e 2 potrebbe essere la diversa distribuzione spaziale, se si considera trascurabile la sovrapposizione spaziale fra i pacchetti d'onda corrispondenti ai due stati, per cui effettivamente $\left<\psi_1\middle|\psi_2\right>\approx 0$.

#### 1.2. Annotated Basic State

Tuttavia, esistono degli altri gradi di libertà che permettono di caratterizzare ulteriormente i fotoni, rendendoli indistinguibili anche quando si trovano nello *stesso* modo. I due gdl standard sono:
- `P` credo polarizzazione sul piano perpendicolare alla direzione del fascio, quindi orizzontale, `H`, o verticale, `V` (https://perceval.quandela.net/docs/polarization.html#polarization)
- `t` used in time circuit with an integer value is defining the period from where the photon is generated (default 0 meaning that the photon is coming from current period).

Gli stati in cui questi gradi di libertà vengono indicati sono detti **Annotated Basic States**

For instance the following are representing different BasicStates with 2 photons having polarization annotations (here limited to `H`/`V`:

- `|{P:H},{P:V}>` corresponding to an annotated `|1,1>` BasicState.

- `|2{P:H},0>` corresponding to an annotated `|2,0>` BasicState where the 2 photons in the first mode are annotated with the same annotation.

- `|{P:H}{P:V},0>` corresponding also to an annotated `|2,0>` BasicState where the two photons in the first mode have different annotations.

- `|1,{t:-1}>` corresponding to an annotated `|1,1>` BasicState, the degree of freedom being time, with the photon in mode 2 coming from previous period, and the photon in mode 1 is not annotated.

Polarizzazioni più generali sono indicate mediante gli angoli $(\phi,\theta)$ che caratterizzano il corrispondente vettore di Jones
$$\vec J = \begin{pmatrix}\cos\frac{\theta}{2}\\e^{i\phi}\sin\frac{\theta}{2}\end{pmatrix}\ .$$
Gli esempi più di base sono i seguenti (immaginando di aver importato SymPy con il comando `import sympy as sp`, per usare la funzione `sp.pi`):

- `|{P:(0,0)}>`, orizzontale, che equivale a `|{P:H}>`
- `|{P:(sp.pi,0)}>`, verticale, che equivale a `|{P:V}>`
- `|{P:(sp.pi/2,0)}>`, diagonale, che equivale a `|{P:D}>`
- `|{P:(sp.pi/2,sp.pi)}>`, anti-diagonale, che equivale a `|{P:A}>`
- `|{P:(sp.pi/2,sp.pi/2)}>`, sinistrorsa, che equivale a `|{P:L}>`
- `|{P:(sp.pi/2,3*sp.pi/2)}>`, destrorsa, che equivale a `|{P:R}>`

Sembra in generale una buona idea rivedersi i vettori di Jones...

#### 1.3. State Vector

Possiamo anche costruire sovrapposizioni di *Basic States* (eventualmente *Annotated*), definendo *State Vectors* come avremmo fatto con `[Annotated]BasicState`, ma usando `StateVector` e sommandoli come se niente fosse.

Basta sommare gli stati in maniera che i rapporti fra i coefficienti della sovrapposizione siano quelli desiderati. Se la norma risulta diversa da 1, il vettore viene automaticamente rinormalizzato, dividendo tutti i coefficienti per la radice quadrata della norma.

Per esempio possiamo costruire uno stato tipo
$$\frac{\left|\phi_1\right>+\sqrt{2}\left|\phi_1\otimes\phi_2\right>}{\sqrt{3}}$$
come segue:

In [8]:
single = pcvl.StateVector("|1,0>")
double = pcvl.StateVector("|1,1>")
total = single + sqrt(2)*double
print(total)

sqrt(3)/3*|1,0>+sqrt(6)/3*|1,1>


o, sempre pensando a due canali, anche uno stato tipo
$$\frac{\left|\phi_1^H\otimes\phi_1^H\right>+\sqrt{2}\left|\phi_1^H\otimes\phi_1^V\right>}{\sqrt{3}}\ ,$$
con 2 fotoni nel primo canale e il secondo canale vuoto, ma con polarizzazioni diverse, scrivendo come segue:

In [10]:
single = pcvl.StateVector("|2{P:H},0>")
double = pcvl.StateVector("|{P:H}{P:V},0>")
total = single + sqrt(2)*double
print(total)

sqrt(3)/3*|2{P:H},0>+sqrt(6)/3*|{P:H}{P:V},0>


#### 1.4 Miscele statistiche e distribuzioni temporali di stati

Informazioni su: https://perceval.quandela.net/docs/states.html#state-vector-distribution

## 2. Parte cicciosa

### 2.1. Circuito inutile ma istruttivo

Importato ciò che ci serve, cominciamo costruendo un semplice circuito con due fasci di fotoni e null'altro, giusto per capire come sono espressi gli stati:

In [16]:
c = pcvl.Circuit(2)
pcvl.pdisplay(c)

#### 2.1.1. Simulazione di evoluzione di uno stato

Costruiamo lo stato di partenza, che avrà:
- 0 fotoni nel *branch* 0;
- 1 fotone nel *branch* ("lower left branch" nel tutorial) 1;

uno stato che in totale ha quindi 1 fotone, come si vede dall'ultima riga:

In [24]:
input_state = pcvl.BasicState("|0,1>")
input_state.n

1

Definiamo un simulatore per il circuito:

In [276]:
backend = pcvl.BackendFactory().get_backend()
simulator = backend(c)

Facciamo evolvere il nostro stato secondo il circuito e, con grande sorpresa, vediamo che lo stato finale è lo stesso che avevamo in principio.

In [277]:
output_state = simulator.evolve(input_state)
print(output_state)

|2,0>


#### 2.1.2. Simulazione di evoluzione di uno stato ed esecuzione di misure

Adesso proviamo a fare proprio 10 conteggi in uscita, per vedere cosa viene (il risultato ci sbalordirà):

In [279]:
ca = pcvl.CircuitAnalyser(simulator,
                         [pcvl.BasicState([0,1])], # lista degli stati in input
                         "*", # tutti i possibili output
                         )
pcvl.pdisplay(ca)

,"|1,0>","|0,1>"
"|0,1>",0,1


che è carino perché è molto semplice, e viene fuori già formattato bene.

Per pigrizia, definiamo le funzioni che simulano l'evoluzione di stati, dato un circuito, e ne restituiscono rispettivamente lo stato finale e la statistica:

In [323]:
def evol_etats_selon_circ(etats, circ):
    outs = []
    for stato in etats:
        backend = pcvl.BackendFactory().get_backend()
        simulator = backend(circ)
        output_state = simulator.evolve(stato)
        print(output_state)
        outs.append(output_state)
    return outs

def stat_etats_selon_circ(etats, circ, outs = "*"):
    backend = pcvl.BackendFactory().get_backend()
    simulator = backend(circ)
    ca = pcvl.CircuitAnalyser(simulator,
                         etats, # lista degli stati in input
                         outs, # tutti i possibili output
                         )
    pcvl.pdisplay(ca)

##### Cella per me

In [71]:
output_counts = {}
for _ in range(10):
    
    # facciamo il conteggio e trasformiamo lo stato in stringa,
    # perché viene meglio la visualizzazione del dizionario
    outcts = str(simulator.sample(input_state))
    
    # se è già stato ottenuto, outcts è presente nelle chiavi del dizionario
    
    # allora basta aumentare il suo conteggio di 1
    # altrimenti, aggiungiamo outcts alle chiavi del dizionario
    # e mettiamo il suo valore a 1
    
    # ma lo faccio in una maniera che trovo carina,
    # e che siccome non ha IF potrebbe essere più efficiente
    # il fatto di aver usato OR fa sì che non crei problemi
    # è tuttavia possibile che il typecasting
    # (la conversione numero <-> logical)
    # complichi la computazione. Chissà
    
    fact = outcts in output_counts.keys()
    output_counts[outcts] = \
        (not fact)*1 or (fact)*(output_counts[outcts]+1)
    
print(output_counts)

{'|0,1>': 10}


### 2.2. Primo esperimento di interferenza

Costruiremo il circuito per blocchi anche se non è necessario, così da poter usare delle idee che saranno utili per la costruzione di circuiti più complessi.

Costruiamo un "blocco computazionale" formato da un solo beam splitter:
- due phasci di fotoni (2 qubit)
- due beam splitter che agiscono sul 1⁰ e sul 2⁰ phascio (indicizzati con 0 e 1)

e poi lo visualizziamo, in forma di circuito e matrice

#### 2.2.1. Con `perceval.lib.phys`:

In [12]:
#creiamo il blocco con un beam splitter
phin = pcvl.Circuit(2)
phin.add((0,1), phys.BS())

#visualizziamo la sua espressione come matrice
pcvl.pdisplay(phin.U)

#visualizziamo il circuito "phys"
pcvl.pdisplay(phin)

Ora i passi già visti nel caso banale:
1. definiamo il simulatore;
2. generiamo le simulazioni e il conteggio con i seguenti stati di partenza:
    - 1 fotone nel ramo superiore
    - 1 fotone nel ramo inferiore
    - 2 fotone nel ramo superiore
    - 1 fotone nel ramo superiore e 1 in quello inferiore
    - 2 fotone nel ramo inferiore.

In [230]:
stat_etats_selon_circ([pcvl.BasicState([0,1]),pcvl.BasicState([1,0]),
                    pcvl.BasicState([2,0]),pcvl.BasicState([1,1]),pcvl.BasicState([0,2])],
                    phin,
                     )

,"|1,0>","|0,1>","|2,0>","|1,1>","|0,2>"
"|0,1>",1/2,1/2,0,0,0
"|1,0>",1/2,1/2,0,0,0
"|2,0>",0,0,1/4,1/2,1/4
"|1,1>",0,0,1/2,0,1/2
"|0,2>",0,0,1/4,1/2,1/4


#### 2.2.2. Con `perceval.lib.symb`:

In [231]:
#creiamo il blocco con un beams splitter
syin = pcvl.Circuit(2)
syin.add((0,1), symb.BS())

#visualizziamo la sua espressione come matrice
pcvl.pdisplay(syin.U)

#visualizziamo il circuito "symb"
pcvl.pdisplay(syin)

In [232]:
stat_etats_selon_circ([pcvl.BasicState([0,1]),pcvl.BasicState([1,0]),
                    pcvl.BasicState([2,0]),pcvl.BasicState([1,1]),pcvl.BasicState([0,2])],
                    syin,
                     )

,"|1,0>","|0,1>","|2,0>","|1,1>","|0,2>"
"|0,1>",1/2,1/2,0,0,0
"|1,0>",1/2,1/2,0,0,0
"|2,0>",0,0,1/4,1/2,1/4
"|1,1>",0,0,1/2,0,1/2
"|0,2>",0,0,1/4,1/2,1/4


Nota: le matrici sono diverse. Tutti i termini hanno lo stesso modulo quadro, ma danno lo stesso risultato. Ciò non dovrebbe essere problematico, perché la differenza fra le trasformazioni che descrivono potrebbe essere esprimibile in termini di fasi *overall*.

Abbiamo generato un po' di statistica, OK, e quello che vediamo è che nel caso a una particella il beam splitter mescola i modi gli uni con gli altri, mentre nel caso a due particelle emergono delle interferenze fra i diversi stati, dovuti alle fasi che il beam splitter introduce sugli stati.

### 2.3. Interferenza e distinguibilità con beam splitter simmetrico

Nel corso di Advanced Quantum Mechanics abbiamo visto cosa succede se si cerca di far passare coppie di particelle quantistiche indistinguibili o rese distinguibili attraverso un beam splitter. La cosa carina è che il comportamento cambia fra i due casi.

Direi che ci sono due modi per rendere distinguibili due particelle identiche che passano nello stesso circuito:
- separarle molto spazialmente, mettendole in due diversi canali o, se devono passare nelle stesse zone di spazio (come per es. lo stesso beam splitter) facendoglielo attraversare a tempi "sufficientemente" distanti;
- aggiungere ulteriori gradi di libertà in cui non vi sia dinamica.

In questo caso, scegliamo la seconda opzione e proviamo a rendere distinguibili 2 fotoni che attraversano lo stesso spazio polarizzandoli con polarizzazioni `H` e `V`.

#### 2.3.1. Costruzione del circuito

Costruiamo un circuito molto semplice, con un solo beam splitter, a valle del quale facciamo misure.

In [233]:
inter = pcvl.Circuit(2)
inter.add((0,1), symb.BS())
pcvl.pdisplay(inter)

#### 2.3.2 Simulazione dell'esperimento

##### 2.3.2.1. Bosoni resi distinguibili

Definiamo uno stato a 2 modi con 2 fotoni, uno per modo, dando ai due polarizzazioni ortogonali, in maniera che siano caratterizzabili mediante questo gdl anche quando sono sovrapposti:

In [118]:
distin = pcvl.AnnotatedBasicState("|{P:H},{P:V}>")

La questione un po' sottile è che se matematicamente pensiamo di rappresentare la posizione spaziale con uno stato, poi vogliamo anche sapere quale fotone ha quale stato, ma non possiamo dire se nel modo 1 ci sia il 1° o il 2° fotone, quindi ci tocca scrivere lo stato come
$$\left|\Psi\right> = \frac{\left|\psi_1^H\otimes\psi_2^V\right> + \left|\psi_2^V\otimes\psi_1^H\right>}{\sqrt 2} = \hat S\left|\psi_1^H\otimes\psi_2^V\right> \ ,$$
a meno di una fase overall, dove definiamo $\hat S$ come l'operatore che simmetrizza lo stato su cui agisce, così ci semplifichiamo la scrittura più tardi.

Esprimiamo la trasformazione imposta dal beam splitter rispetto al gdl spaziale come:
$$\hat U_{BS} = \frac{\left|\psi_1\middle>\middle<\psi_1\right| + \left|\psi_1\middle>\middle<\psi_2\right| + \left|\psi_2\middle>\middle<\psi_1\right| - \left|\psi_2\middle>\middle<\psi_2\right|}{2}\ ,$$
dove i coefficienti vengono dal fatto di aver preso l'espressione data dalla versione **phys**.

A valle del beam splitter, lo stato risulta essere il seguente:
$$\hat U_{BS}\otimes\hat U_{BS}\left|\Psi\right> = \frac{\hat S\left|\psi_1^H\otimes\psi_1^V\right> - \hat S\left|\psi_2^H\otimes\psi_2^V\right> + \hat S\left|\psi_2^H\otimes\psi_1^V\right> - \hat S\left|\psi_1^H\otimes\psi_2^V\right>}{2}$$

Costruiamo il simulatore e simuliamo l'evoluzione dello stato e otteniamo la statistica del conteggio dei fotoni nei due modi:

In [234]:
stat_etats_selon_circ([distin],inter)

,"|2,0>","|1,1>","|0,2>"
"|{P:H},{P:V}>",1/4,1/2,1/4


Vediamo che in effetti la statistica è quella di due particelle indipendenti che vengono "smistate" con 50% di probabilità in un canale e 50% nell'altro, indipendentemente.

##### 2.3.2.2. Bosoni indistinguibili

Definiamo uno stato a 2 modi con 2 fotoni, uno per canale, nei seguenti modi:
1. non specificando la polarizzazione.
2. dando ai due la stessa polarizzazione;

(nell'ordine inverso non so ancora perché ma non mi funziona)

Se Perceval non introduce gdl non specificati dall'utente, ci aspettiamo lo stesso risultato.

In [124]:
indist1 = pcvl.AnnotatedBasicState("|1,1>")
indist2 = pcvl.AnnotatedBasicState("|{P:H},{P:H}>")

Ri-costruiamo il simulatore (se non lo faccio, per qualche stato crea problemi, boh) e simuliamo l'evoluzione dello stato e otteniamo la statistica del conteggio dei fotoni nei due modi:

In [235]:
stat_etats_selon_circ([indist1,indist2],inter)

,"|2,0>","|1,1>","|0,2>"
"|1,1>",1/2,0,1/2
"|{P:H},{P:H}>",1/2,0,1/2


In questo caso emergono delle interferenze che si collegano al fatto che lo stato in ingresso non ha il gdl delle polarizzazioni. Infatti, ricordiamo che lo stato in uscita *con* le polarizzazioni era

$$\hat U_{BS}\otimes\hat U_{BS}\left|\Psi\right> = \frac{\hat S\left|\psi_1^H\otimes\psi_1^V\right> - \hat S\left|\psi_2^H\otimes\psi_2^V\right> + \hat S\left|\psi_2^H\otimes\psi_1^V\right> - \hat S\left|\psi_1^H\otimes\psi_2^V\right>}{2}\ .$$

Se rimuoviamo l'indice sulla polarizzazione, in due stati si trova degenerazione, pertanto
$$\hat S\left|\psi_1^H\otimes\psi_1^V\right> = \frac{\left|\psi_1^H\otimes\psi_1^V\right> + \left|\psi_1^V\otimes\psi_1^H\right>}{\sqrt 2} \rightarrow$$
$$\rightarrow\frac{\left|\psi_1\otimes\psi_1\right> + \left|\psi_1\otimes\psi_1\right>}{\sqrt 2} = \sqrt 2 \left|\psi_1\otimes\psi_1\right>\ .$$

Il risultato è che quando si esegue l'esperimento senza il gdl aggiuntivo della polarizzazione si trova
$$\hat U_{BS}\otimes\hat U_{BS}\left|\Psi\right> = \frac{\sqrt2\left|\psi_1\otimes\psi_1\right> - \sqrt2\left|\psi_2\otimes\psi_2\right> + \hat S\left|\psi_2\otimes\psi_1\right> - \hat S\left|\psi_1\otimes\psi_2\right>}{2}=\frac{\sqrt2\left|\psi_1\otimes\psi_1\right> - \sqrt2\left|\psi_2\otimes\psi_2\right>}{2}=\frac{\left|\psi_1\otimes\psi_1\right> - \left|\psi_2\otimes\psi_2\right>}{\sqrt2}\ .$$

###### Considerazioni sulla simmetria e sulla filosofia

Se decidessimo che la prima particella è quella che si trova all'inizio nello stato $\left|\psi_1\right>$ e la seconda quella in $\left|\psi_2\right>$ ci troveremmo presto a vedere che comunque "non sappiamo più quale particella è quale" e troveremmo in output lo stato
$$\hat U_{BS}\otimes\hat U_{BS}\left|\psi_1\otimes\psi_2\right> = \frac{\left|\psi_1\otimes\psi_1\right> + \left|\psi_2\otimes\psi_2\right> + \left|\psi_2\otimes\psi_1\right> - \left|\psi_1\otimes\psi_2\right>}{2}\ ,$$
che sarebbe problematico per una coppia di *bosoni* che a questo punto è stata mescolata, perché ha una parte simmetrica e una antisimmetrica. Quindi in un certo senso, l'ammissione che le due particelle hanno per almeno un istante perso la loro identità richiede che non l'abbiano mai avuta e che lo stato "giusto" sia sempre stato quello simmetrico $\hat S\left|\psi_1\otimes\psi_2\right>$.

##### 2.3.2.2. Fermioni indistinguibili

Vorrei trovare un modo di simulare la statistica dei fermioni con stati bosonici. L'idea è che magari introducendo fasi e giocando con i gdl si riesce a trovare una cosa che agisca su stati bosonici come se fosse un beam splitter e gli stati fossero in realtà fermionici. Un po' come gli atomi, che possono avere comportamento efficace di fermioni o di bosoni in base a quanti fermioni spin-1/2 li compongono. Fun fact: non ha ancora funzionato.